## Data sourcing

Source data from various source systems and ingest them using python code.

1. Parquet files
2. CSV files
3. APIs
4. RDBMS databases
5. HTML

In [1]:
# import modules
import certifi
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import urllib3
from urllib3 import request
import requests
from unicodedata import normalize

### Sourcing Parquet data

Please visit the url https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
# Read data from the Parquet file. We use pandas read_parquet method for ease and speed.
df_parquet = pd.read_parquet("yellow_tripdata_2022-01.parquet")
df_parquet.head()

### Sourcing CSV data 

Please visit the url https://data.cityofnewyork.us/resource/h9gi-nx95.csv?$limit=500


In [ ]:
# Read data from the CSV file. We use pandas read_csv method for ease and speed.
df_csv = pd.read_csv("h9gi-nx95.csv")
df_csv.head()

### Sourcing data from APIs

Please make sure to install the certifi library using - pipenv install certifi

In [ ]:
# get api data from url
url = 'https://data.cityofnewyork.us/resource/h9gi-nx95.json?$limit=500'

# Check if API is available to retrive the data
apt_status = http.request('GET', url).status
print(apt_status)
if apt_status == 200:
    # Sometimes we get certificate error . We shoul never silence this error as this may cause a securirty threat.
    # Create a Pool manager that can be used to read the API response 
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
    data = json.loads(http.request('GET', url).data.decode('utf-8'))
    df_api = pd.json_normalize(data)
else:
    df_api = pd.Dataframe()
df_api.head(10)

In [2]:
import sys
import requests
import json
import logging

logging.captureWarnings(True)

##
##    function to obtain a new OAuth 2.0 token from the authentication server
##
def get_new_token():

    auth_server_url = "https://auth-pisa-qa.staging.gcp-eu.taocloud.org/v1/oauth2/tokens"
    client_id = 'Q2caBAhwvNF8NgmrEPKrUw'
    client_secret = 'wHtKRe6eA7BWRBNIISiTQk3uM210OJdQ'

    token_req_payload = {'grant_type': 'client_credentials'}

    token_response = requests.post(auth_server_url,
    data=token_req_payload, verify=False, allow_redirects=False,
    auth=(client_id, client_secret))
                
    if token_response.status_code !=200:
        print("Failed to obtain token from the OAuth 2.0 server", file=sys.stderr)
        sys.exit(1)

    print("Successfuly obtained a new token")
    tokens = json.loads(token_response.text)
    return tokens['access_token']

## 
## 	obtain a token before calling the API for the first time
##
token = get_new_token()


Successfuly obtained a new token


In [3]:
# Get request function

def req_get(url,token,params=None,data=None):

##
##   call the API with the token
##
    
    api_call_headers = {'Authorization': 'Bearer ' + token}
    api_call_response = requests.get(url, params, headers=api_call_headers, json=data, verify=False, timeout = 60)

    if	api_call_response.status_code == 401 | api_call_response.status_code == 403:
        token = get_new_token()

    data = json.loads(api_call_response.text)
    df_api = pd.json_normalize(data)
    return df_api

# Get request function

def req_post(url,token,params=None,data=None):

##
##   call the API with the token
##
    
    api_call_headers = {'Authorization': 'Bearer ' + token}
    api_call_response = requests.post(url, params, headers=api_call_headers, json=data, verify=False, timeout = 60)

    if	api_call_response.status_code == 401 | api_call_response.status_code == 403:
        token = get_new_token()

    data = json.loads(api_call_response.text)
    df_api = pd.json_normalize(data)
    return df_api


In [4]:
api_url_health = "https://dynamic-query-api-pisa-qa.staging.gcp-eu.taocloud.org/api/v1/health"
df_health = req_get(url=api_url_health,token=token)
df_health

,status
0,ok


In [6]:
api_url_entity_list = "https://dynamic-query-api-pisa-qa.staging.gcp-eu.taocloud.org/api/v1/entity/list"
df_entity_list = req_get(url=api_url_entity_list,token=token)
df_entity_list

,name,description
0,portalSessions,Holds information on portal sessions
1,portalUserGroups,Holds information on portal user groups
2,portalGroups,Holds information on portal groups
3,portalUserSessions,Holds information on portal user sessions
4,portalUser,Holds information on portal user
5,portalBattery,Holds information on portal battery
6,testRunnerDeliveries,Holds information on test runner deliveries
7,datastoreDeliveries,Holds information on datastore deliveries
8,datastoreDeliveryResults,Holds information on datastore delivery results
9,datastoreItemResults,Holds information on datastore item results


In [47]:
entityId = df_entity_list.iloc[8]['name']
entityId
api_url_del_results = "https://dynamic-query-api-pisa-qa.staging.gcp-eu.taocloud.org/api/v1/search/%s" % entityId
data = {
  'pageSize': 1,
  'pageNumber': 0,
  'filters': [
    {
      'field': 'testQtiId',
      'type': 'terms',
      'values': ['R432']
    }
  ]
}

df_del_results_all = req_post(url=api_url_del_results,token=token, data = data)


In [58]:
df_del_results = pd.json_normalize(df_del_results_all.iloc[0]['data'][0])
df_del_results.shape
df_del_results.head(10)

,sessionEndTime,metadataRaw,login,testQtiId,testQtiTitle,submissionTime,duration,score,deliveryId,isDeleted,...,items.item-1.qtiLabel,items.item-1.duration,items.item-1.score,items.item-1.itemEndTime,items.item-1.outcomes.SCORE,items.item-1.outcomes.completionStatus,items.item-1.responses.RESPONSE.correct,items.item-1.responses.RESPONSE.value,items.item-1.completionStatus,items.item-1.itemStartTime
0,1699421042000,"[{'propertyLabel': 'PISA25 Languages', 'values...",VS02,R432,About A Book (R432),1699421042000,32,0,9b52d97a6ea5,False,...,CR432Q01,6,0,1699421021000,0,completed,False,"{NULL; 8; ''; 6241; 6328; 0; 0; '<div class=""R...",completed,1699421012000


In [64]:
df_del_results_meta = pd.json_normalize(df_del_results_all.iloc[0]['data'][0]['metadataRaw'])
df_del_results_meta

,propertyLabel,values,alias,propertyUri,type
0,PISA25 Languages,"[{'vuri': 'https://www.oecd.org/en-ZZ', 'vlabe...",None,https://pisa2025.eu.premium.taocloud.org/#i63b...,http://www.tao.lu/datatypes/WidgetDefinitions....
1,PISA25 Is Translated Unit?,[{'vuri': 'http://www.tao.lu/Ontologies/generi...,pisa25_is_translated_unit,https://pisa2025.eu.premium.taocloud.org/#i63b...,http://www.tao.lu/datatypes/WidgetDefinitions....
2,PISA25 Source Unit State,[{'vuri': 'https://www.oecd.org/translationSta...,pisa25_sourse_unit_state,https://pisa2025.eu.premium.taocloud.org/#i63b...,http://www.tao.lu/datatypes/WidgetDefinitions....
3,PISA25 Domains,"[{'vuri': 'https://www.oecd.org/REA', 'vlabel'...",pisa25_domains,https://pisa2025.eu.premium.taocloud.org/#i63e...,http://www.tao.lu/datatypes/WidgetDefinitions....
4,PISA25 Group,"[{'vuri': 'batch1', 'vlabel': None}]",pisa25_group,https://pisa2025.eu.premium.taocloud.org/#i641...,http://www.tao.lu/datatypes/WidgetDefinitions....
5,UI Engine,[{'vuri': 'testRunnerConfiguration_ui-engine:c...,ui-engine,https://pisa2025.eu.premium.taocloud.org/#i649...,http://www.tao.lu/datatypes/WidgetDefinitions....
6,Label,"[{'vuri': 'About A Book (R432)', 'vlabel': None}]",None,http://www.w3.org/2000/01/rdf-schema#label,http://www.tao.lu/datatypes/WidgetDefinitions....


In [72]:
df_del_results_meta_vals = pd.json_normalize(df_del_results_all.iloc[0]['data'][0]['metadataRaw'],record_path='values',meta=['propertyLabel','alias','propertyUri','type'])
df_del_results_meta_vals

,vuri,vlabel,propertyLabel,alias,propertyUri,type
0,https://www.oecd.org/en-ZZ,English (ZZ),PISA25 Languages,None,https://pisa2025.eu.premium.taocloud.org/#i63b...,http://www.tao.lu/datatypes/WidgetDefinitions....
1,http://www.tao.lu/Ontologies/generis.rdf#False,False,PISA25 Is Translated Unit?,pisa25_is_translated_unit,https://pisa2025.eu.premium.taocloud.org/#i63b...,http://www.tao.lu/datatypes/WidgetDefinitions....
2,https://www.oecd.org/translationStateTranslated,Translated,PISA25 Source Unit State,pisa25_sourse_unit_state,https://pisa2025.eu.premium.taocloud.org/#i63b...,http://www.tao.lu/datatypes/WidgetDefinitions....
3,https://www.oecd.org/REA,REA,PISA25 Domains,pisa25_domains,https://pisa2025.eu.premium.taocloud.org/#i63e...,http://www.tao.lu/datatypes/WidgetDefinitions....
4,batch1,None,PISA25 Group,pisa25_group,https://pisa2025.eu.premium.taocloud.org/#i641...,http://www.tao.lu/datatypes/WidgetDefinitions....
5,testRunnerConfiguration_ui-engine:core1,Core 1,UI Engine,ui-engine,https://pisa2025.eu.premium.taocloud.org/#i649...,http://www.tao.lu/datatypes/WidgetDefinitions....
6,About A Book (R432),None,Label,None,http://www.w3.org/2000/01/rdf-schema#label,http://www.tao.lu/datatypes/WidgetDefinitions....


In [37]:
deliveryId = "4d31793696fc"
api_url_delid_result = "https://dynamic-query-api-pisa-qa.staging.gcp-eu.taocloud.org/api/v1/deliveries/%s/results" % deliveryId
params = (('page',1),('results',10),('deliveryId',deliveryId))
# df_results_delid = req_get(url=api_url_results_all,token=token, params = params)
# df_results_delid

'[["page", 1], ["results", 10], ["deliveryId", "4d31793696fc"]]'

### Sourcing Data from RDBMS tables

In [ ]:
# Read sqlite query results into a pandas DataFrame
with sqlite3.connect("movies.sqlite") as conn:
    df = pd.read_sql("SELECT * from movies", conn)
df.head()

# Sourcing data from Webpages

Please visit the url https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)

In [ ]:
# get data from url
df_html = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)',match = 'by country')
# Let's see how many tables are there with tage ' by county'
print(len(df_html)) # There are 4 tables
# Let's see the first table
df_html[0]